In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import os

In [2]:
X_train = pd.read_csv("../data/processed/movies_X_train_raw_imputed.csv")
X_test = pd.read_csv("../data/processed/movies_X_test_raw_imputed.csv")
y_train = pd.read_csv("../data/processed/movies_y_train.csv")
y_test = pd.read_csv("../data/processed/movies_y_test.csv")

In [8]:
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer

cvec = CountVectorizer(min_df=0.01)

cvec.fit(train_data['genres'])

train_data = pd.concat([train_data.drop(columns=['genres']), pd.DataFrame(cvec.transform(train_data['genres']).todense(), columns="genre_"+ cvec.get_feature_names_out())], axis=1)
test_data = pd.concat([test_data.drop(columns=['genres']), pd.DataFrame(cvec.transform(test_data['genres']).todense(), columns="genre_"+ cvec.get_feature_names_out())], axis=1)

In [5]:
#Try lemmatizing and BOW
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]
        
class StemTokenizer(object):
    def __init__(self):
        self.stemmer = SnowballStemmer('english')
    def __call__(self, articles):
        return [self.stemmer.stem(t) for t in word_tokenize(articles)]


tf_vectorizer = TfidfVectorizer(tokenizer=LemmaTokenizer(),
                       strip_accents = 'unicode',
                       stop_words = 'english',
                       lowercase = True,
                        min_df=0.015
                    )
tf_vectorizer.fit(train_data['summary'])


train_data = pd.concat([train_data, pd.DataFrame(tf_vectorizer.transform(train_data['summary']).todense(), columns="summary_"+ tf_vectorizer.get_feature_names_out())], axis=1)
test_data = pd.concat([test_data, pd.DataFrame(tf_vectorizer.transform(test_data['summary']).todense(), columns="summary_"+ tf_vectorizer.get_feature_names_out())], axis=1)

/home/jyl2fb/.conda/envs/netflix_proj/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/jyl2fb/.conda/envs/netflix_proj/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [13]:
save_path = '../models/agModels-predicttop10'

predictor = TabularPredictor(label='is_top10', path=save_path, eval_metric='roc_auc').fit(train_data.drop(columns=['numVotes', 'averageRating']), presets='best_quality')

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=7, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "../models/agModels-predicttop10/"
AutoGluon Version:  0.6.0
Python Version:     3.9.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Aug 7 18:08:02 UTC 2019
Train Data Rows:    733
Train Data Columns: 37
Label Column: is_top10
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:             

In [14]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,0.840791,0.810295,0.213371,0.322723,26.647933,0.213371,0.322723,26.647933,1,True,7
1,WeightedEnsemble_L2,0.825521,0.823407,4.043754,1.545058,108.311694,0.009853,0.001154,0.599551,2,True,14
2,XGBoost_BAG_L1,0.809777,0.799611,1.452227,0.127034,5.713532,1.452227,0.127034,5.713532,1,True,11
3,LightGBM_BAG_L1,0.807528,0.795923,0.095681,0.133713,14.242337,0.095681,0.133713,14.242337,1,True,4
4,RandomForestGini_BAG_L1,0.801136,0.769365,0.109831,0.213605,0.789561,0.109831,0.213605,0.789561,1,True,5
5,LightGBMLarge_BAG_L1,0.790720,0.783640,0.485903,0.087111,46.846742,0.485903,0.087111,46.846742,1,True,13
6,LightGBMXT_BAG_L1,0.788352,0.768172,0.149622,0.170271,14.967913,0.149622,0.170271,14.967913,1,True,3
7,RandomForestEntr_BAG_L1,0.785926,0.771927,0.235239,0.157010,0.799900,0.235239,0.157010,0.799900,1,True,6
8,NeuralNetFastAI_BAG_L1,0.773911,0.748646,2.197144,1.058547,12.063956,2.197144,1.058547,12.063956,1,True,10
9,ExtraTreesGini_BAG_L1,0.770597,0.752461,0.104735,0.195045,0.689928,0.104735,0.195045,0.689928,1,True,8


In [219]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

def get_results_model(model, X_test=X_test, y_test=y_test, name=""):
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test).iloc[:,1]
    print(f"{name} AUC: %0.5f" % roc_auc_score(y_test, y_pred_proba))
    print(f"{name} F1: %0.5f" % f1_score(y_test, y_pred))
    print(f"{name} Accuracy: %0.5f" % accuracy_score(y_test, y_pred))
    return y_pred, y_pred_proba

def get_results_preds(y_labels, y_preds_proba, y_preds,  name=""):

    print(f"{name} AUC: %0.5f" % roc_auc_score(y_labels, y_preds_proba))
    print(f"{name} F1: %0.5f" % f1_score(y_labels, y_preds))
    print(f"{name} Accuracy: %0.5f" % accuracy_score(y_labels, y_preds))

In [221]:
y_pred, y_pred_proba = get_results_model(predictor, test_data, test_data['is_top10'], name="Ensemble Model")

Ensemble Model AUC: 0.80007
Ensemble Model F1: 0.75862
Ensemble Model Accuracy: 0.73370


In [72]:
save_path = "../models/multimodal"
predictor2 = TabularPredictor(label='is_top10', path=save_path, eval_metric='roc_auc', )
predictor2.fit(train_data.drop(columns=['numVotes', 'averageRating']), presets='best_quality')

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=7, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "../models/multimodal/"
AutoGluon Version:  0.6.0
Python Version:     3.9.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Aug 7 18:08:02 UTC 2019
Train Data Rows:    733
Train Data Columns: 37
Label Column: is_top10
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    6311

In [74]:
predictor2.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,0.840791,0.810295,0.138211,0.582095,114.730787,0.138211,0.582095,114.730787,1,True,7
1,WeightedEnsemble_L2,0.830137,0.820170,16.917832,2.665285,508.584302,0.007935,0.000591,0.630841,2,True,14
2,XGBoost_BAG_L1,0.809777,0.799611,0.154470,0.179758,23.262070,0.154470,0.179758,23.262070,1,True,11
3,LightGBM_BAG_L1,0.807528,0.795923,0.087427,0.147339,42.296840,0.087427,0.147339,42.296840,1,True,4
4,RandomForestGini_BAG_L1,0.801136,0.769365,0.169875,0.166012,0.989191,0.169875,0.166012,0.989191,1,True,5
5,LightGBMLarge_BAG_L1,0.790720,0.783640,0.183929,0.140950,166.475157,0.183929,0.140950,166.475157,1,True,13
6,LightGBMXT_BAG_L1,0.788352,0.768172,0.121910,0.158409,43.211514,0.121910,0.158409,43.211514,1,True,3
7,RandomForestEntr_BAG_L1,0.785926,0.771927,0.250696,0.165139,0.911539,0.250696,0.165139,0.911539,1,True,6
8,NeuralNetFastAI_BAG_L1,0.773911,0.748646,15.654653,0.747402,18.472049,15.654653,0.747402,18.472049,1,True,10
9,ExtraTreesGini_BAG_L1,0.770597,0.752461,0.344684,0.171485,0.807926,0.344684,0.171485,0.807926,1,True,8


In [84]:
#Try lemmatizing and BOW
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from nltk.stem import SnowballStemmer
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]
        
class StemTokenizer(object):
    def __init__(self):
        self.stemmer = SnowballStemmer('english')
    def __call__(self, articles):
        return [self.stemmer.stem(t) for t in word_tokenize(articles)]


tf_vectorizer = CountVectorizer(tokenizer=LemmaTokenizer(),
                       strip_accents = 'unicode',
                       stop_words = 'english',
                       lowercase = True,
                       min_df=0.04
                    )
tf_vectorizer.fit(train_data['summary'])

/home/jyl2fb/.conda/envs/netflix_proj/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


CountVectorizer(min_df=0.04, stop_words='english', strip_accents='unicode',
                tokenizer=<__main__.LemmaTokenizer object at 0x2aacc4b39850>)

In [87]:
save_path = "../models/tfidf"
predictor2 = TabularPredictor(label='is_top10', path=save_path, eval_metric='roc_auc', )
predictor2.fit(train_data.drop(columns=['numVotes', 'averageRating']))

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=7, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "../models/tfidf/"
AutoGluon Version:  0.6.0
Python Version:     3.9.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Aug 7 18:08:02 UTC 2019
Train Data Rows:    733
Train Data Columns: 58
Label Column: is_top10
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    62833.39 

In [88]:
predictor2.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,0.845289,0.807208,0.105034,0.542356,119.745394,0.105034,0.542356,119.745394,1,True,7
1,WeightedEnsemble_L2,0.834635,0.821266,2.196087,2.705328,509.994133,0.008023,0.000723,0.639256,2,True,14
2,LightGBM_BAG_L1,0.807528,0.795923,0.214329,0.151193,47.043171,0.214329,0.151193,47.043171,1,True,4
3,XGBoost_BAG_L1,0.805161,0.795633,0.154573,0.135750,23.602797,0.154573,0.135750,23.602797,1,True,11
4,RandomForestGini_BAG_L1,0.794626,0.758487,0.164542,0.166851,0.960546,0.164542,0.166851,0.960546,1,True,5
5,RandomForestEntr_BAG_L1,0.793265,0.765151,0.360969,0.165040,1.003458,0.360969,0.165040,1.003458,1,True,6
6,LightGBMLarge_BAG_L1,0.791075,0.789107,0.149165,0.142471,173.960655,0.149165,0.142471,173.960655,1,True,13
7,LightGBMXT_BAG_L1,0.790009,0.771714,0.138928,0.116605,42.886098,0.138928,0.116605,42.886098,1,True,3
8,ExtraTreesEntr_BAG_L1,0.776278,0.761720,0.622480,0.168473,0.851739,0.622480,0.168473,0.851739,1,True,9
9,NeuralNetFastAI_BAG_L1,0.770478,0.758610,0.811756,0.838664,19.285250,0.811756,0.838664,19.285250,1,True,10
